# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [ ]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
from matplotlib import pyplot as plt


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    # Returns 2nd PPG channel and all three ACC channels
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(np.array(errs))
    confs = np.hstack(np.array(confs))
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Performs activities comprising the signal analysis pipeline:
        Filter Noise -> Train Heuristic -> Predict PR -> Compute Confidence
    
    Args:
        data_fl: Name of the .mat files that contain signal data
        ref_fl: Name of the .mat files that contain reference data

    Returns:
        numpy arrays for pulse rate errors and confidences.
    """
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_pr = sp.io.loadmat(ref_fl)['BPM0']

    # Define initial values
    fs = 125                                     # Part 1 README.md
    low_freq, high_freq = 40/60, 240/60          # TroikaDataset README.pdf
    errors, confidence = np.array([]), np.array([])

    # Apply bandpass filter to PPG and ACC signal to remove noise
    ppg_filtered = BandpassFilter(ppg, fs, low_freq, high_freq)
    accx_filtered = BandpassFilter(accx, fs, low_freq, high_freq)
    accy_filtered = BandpassFilter(accy, fs, low_freq, high_freq)
    accz_filtered = BandpassFilter(accz, fs, low_freq, high_freq)

    # Train on 8 second windows with 2 second overlaps to match ground-truth measurements
    window_length_s = 8
    window_shift_s = 2
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    end_index = min(len(ppg), len(ref_pr) * fs * 2)
    est_last = 0
    
    for i in range(0, (end_index - window_length), window_shift):
        
        # Compute slice of signal
        ppg_window = ppg_filtered[i: i + window_length]
        accx_window = accx_filtered[i: i + window_length]
        accy_window = accy_filtered[i: i + window_length]
        accz_window = accz_filtered[i: i + window_length]

        # Find PPG and ACC peaks (frequencies)
        ppg_peaks_f, ppg_peaks_m, ppg_freq, ppg_fft = ListPeaks(ppg_window, fs)
        accx_peaks_f, _, _, _ = ListPeaks(accx_window, fs)
        accy_peaks_f, _, _, _ = ListPeaks(accy_window, fs)
        accz_peaks_f, _, _, _ = ListPeaks(accz_window, fs)
        
        """
        ## Code for debugging purposes
        if i < 10:
            # Plot PPG signal
            x = ref_pr[int(i/(fs*2))]/60
            y = max(ppg_peaks_m)
            plt.figure(figsize=(12, 8))
            plt.xlim(0, 6)
            plt.plot((x, x), (0, y), 'g-')
            plt.plot(ppg_freq, ppg_fft)
            plt.plot(ppg_peaks_f, ppg_peaks_m, 'r.', ms=10)
            plt.xlabel('Frequency (Hz)')
        """
        
        # Find best estimated pulse rate
        est_pr = FindBestPR(ppg_peaks_f, ppg_peaks_m, accx_peaks_f, accy_peaks_f, accz_peaks_f)
        est_pr_bpm = est_pr * 60                 # Multiplying by 60 to get BPM
    
        # Compute absolute error for pulse rate estimate
        err = np.abs(est_pr_bpm - ref_pr[int(i/(fs*2))])
        
        """
        ## Code for debugging purposes
        # Plot pulse rate estimates with high errors relative to reference pulse rate
        if err > 80:
            # Plot PPG signal
            x_ref = ref_pr[int(i/(fs*2))]/60
            x_pred =est_pr
            y = max(ppg_peaks_m)
            plt.figure(figsize=(12, 8))
            plt.xlim(0, 6)
            plt.plot((x_ref, x_ref), (0, y), 'g-')
            plt.plot(ppg_freq, ppg_fft)
            plt.plot((x_pred, x_pred), (0, y), 'r-')
            plt.xlabel('Frequency (Hz)')
        """
        
        # Use last good estimate if error is high
        if err > 15:
            est_pr = est_last
        else:
            est_last = est_pr

        errors = np.append(errors, err)
    
        # Compute estimation confidence for pulse rate estimate
        conf = CalcConfidence(est_pr, ppg_freq, ppg_fft)
        confidence = np.append(confidence, conf)
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidence

def BandpassFilter(signal, fs, low_freq, high_freq):
    """
    Applies a bandpass filter to a signal.
    
    Args:
        signal: (np.array) The input signal.
        fs: sampling frequency.
        low_freq: low frequency limit in Hz.
        high_freq: high frequency limit in Hz.
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(1, (low_freq, high_freq), btype='bandpass', fs=fs)
    
    return sp.signal.filtfilt(b, a, signal)

def ListPeaks(signal, fs):
    """
    Calculates frequency peaks for signal
    
    Args:
        signal: (np.array) The input signal
        fs: sampling frequency.
        height: height parameter for SCIPY find_peaks function.
        distance: height parameter for SCIPY find_peaks function..

    Returns:
        numpy arrays for signal peak frequencies and FFTs, signal frequencies and FFTs.
    """
    
    # Compute the FFT of the signal
    fft_len = len(signal) * 4                    # 0-pad signal
    freqs = np.fft.rfftfreq(fft_len, 1/fs)
    fft_mag = np.abs(np.fft.rfft(signal, fft_len))
    fft_mag[freqs <= 40/60.0] = 0.0
    fft_mag[freqs >= 240/60.0] = 0.0
    
    # Calculate peak indices in the signal
    #print(max(fft_mag), height)
    peaks_ind = sp.signal.find_peaks(fft_mag)[0]
    
    # Citation: Udacity Knowledge at https://knowledge.udacity.com/questions/224181
    # Sort peak magnitudes strongest to weakest 
    peaks_mag = []
    for i in peaks_ind:
        peaks_mag.append(fft_mag[i])
    peaks_mag = sorted(peaks_mag)[::-1]
    
    # Sort peak frequencies from strongest to weakest magnitude
    peaks_freq = []
    for m in peaks_mag:
        peaks_freq.append(freqs[np.where(fft_mag==m)[0][0]])
    
    return peaks_freq, peaks_mag, freqs, fft_mag

def FindBestPR(ppg_peaks_f, ppg_peaks_m, accx_peaks_f, accy_peaks_f, accz_peaks_f):
    """
    Computes best pulse rate given PPG and ACC peaks
    
    Args:
        ppg_peaks_f: (np.array) Array of PPG signal frequency peaks.
        ppg_freqs_m: (np.array) Array of PPG signal frequency peaks.
        accx_peaks_f: (np.array)  Array of ACC x-channel frequency peaks.
        accy_peaks_f: (np.array)  Array of ACC y-channel frequency peaks.
        accz_peaks_f: (np.array)  Array of ACC z-channel frequency peaks.

    Returns:
        best estimated pulse rate.
    """
    # Initialize selection parameters
    band = 5 / 60                                # BPM divided by 60 for Hz
    threshold = 0.35 * max(ppg_peaks_m)
    
    # Loop through all PPG peaks to identify best one
    for i in range(0, len(ppg_peaks_m)):

        # Consider candidate peak if strong enough
        if ppg_peaks_m[i] > threshold:

            # Select peak if not similar to any dominant ACC channel frequency
            low_freq = ppg_peaks_f[i] - band
            high_freq = ppg_peaks_f[i] + band
            
            accx_test = (accx_peaks_f[0] < low_freq) or (accx_peaks_f[0] > high_freq)
            accy_test = (accy_peaks_f[0] < low_freq) or (accy_peaks_f[0] > high_freq)
            accz_test = (accz_peaks_f[0] < low_freq) or (accz_peaks_f[0] > high_freq)
            
            if (accx_test and accy_test and accz_test):
                return ppg_peaks_f[i]
    
    # Use highest PPG peak as best estimated pulse rate if no alternatives are found    
    return ppg_peaks_f[0]

def CalcConfidence(est_pr, ppg_freqs, ppg_fft):
    """
    Confidence based on signal power around estimated heart rate
    
    Args:
        est_pr: The pulse rate estimate.
        ppg_freqs: (np.array) PPG frequencies.
        ppg_fft: (np.array) PPG FFT magnitudes.

    Returns:
        numpy array for confidence.
    """
    # Define frequency window centered on estimated pulse rate
    freq_width = 40 / 60                         # bpm divided by 60 for Hz
    freq_window = (ppg_freqs > est_pr - freq_width) & (ppg_freqs < est_pr + freq_width)
    
    # Calcuate power of pulse rate signal and the total power
    signal_power = np.sum(ppg_fft[(freq_window)])
    total_power = np.sum(ppg_fft)
    
    # Calculate confidence of pulse rate estimate
    confidence = signal_power / total_power
    
    return confidence